In [ ]:
!pip install gensim

: 

In [31]:
!pip install nltk
!pip install pandas
!pip install sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.5 MB/s eta 0:00:000m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

script = pd.read_csv('Game_of_Thrones_Script_count_senti.csv')
script_text = script[["Name", "Sentence"]]

In [23]:
script_text

,Name,Sentence
0,waymar royce,What do you expect? They're savages. One lot s...
1,will,I've never seen wildlings do a thing like this...
2,waymar royce,How close did you get?
3,will,Close as any man would.
4,gared,We should head back to the wall.
...,...,...
23906,brienne,I think we can all agree that ships take prece...
23907,bronn,I think that's a very presumptuous statement.
23908,tyrion lannister,I once brought a jackass and a honeycomb into ...
23909,man,The Queen in the North!


In [24]:
import nltk
nltk.download('stopwords')

STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    data = data.dropna(how="any")
    
    for col in ['Sentence']:
        data[col] = data[col].apply(clean_sentence)
    
    return data

data = clean_dataframe(script_text)
data.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mdargatz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Name,Sentence
0,waymar royce,expect theyre savages lot steals goat lot they...
1,will,ive wildlings ive life
2,waymar royce,close
3,will,close
4,gared,head back wall


In [25]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for col in ['Sentence']:
        for sentence in data[col].iteritems():
            word_list = sentence[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(data)        
corpus[0:2]

AttributeError: 'Series' object has no attribute 'iteritems'

In [27]:
!pip install pandas==1.5

In [28]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [30]:
# Load Google's pre-trained Word2Vec model.
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [ ]:
tsne_plot(model)

In [ ]:
data = model.most_similar('king', topn=5)

In [ ]:
plot_similarity(data)

In [ ]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'

# set the style of the axes and the text color
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'
plt.rcParams['text.color']='#333F4B'
plt.style.use("fivethirtyeight")
plt.xticks(rotation=45)
plt.plot(data["word"],data["number"]*100, "o", markersize=24, color='#007ACC', alpha=0.9)
plt.bar(data["word"],data["number"]*100, alpha=0.4, width=0.3)

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
model.most_similar(positive=['mother', 'father'], topn=5)

In [ ]:
model_got = word2vec.Word2Vec(corpus, size=64, window=5, min_count=20, workers=8)

In [ ]:
model_got.most_similar("jon")

In [ ]:
model_got.most_similar("stark")

In [ ]:
model_got.most_similar("tywin")

In [ ]:
model_got.most_similar(positive=['tywin', 'son'])

In [ ]:
model_got.most_similar(positive=['jaime', 'cersei', 'tyrion'], topn=5)